In [1]:
from pyspark import *

In [2]:
df_train = spark.read.load("/FileStore/tables/train.txt", format="csv" , sep=" ", inferSchema="true", header="false").drop('_c26','_c27')
df_test = spark.read.load("/FileStore/tables/test.txt", format="csv" , sep=" ", inferSchema="true", header="false").drop('_c26','_c27')
df_truth = spark.read.load("/FileStore/tables/truth.txt", format="csv" , sep=" ", inferSchema="true", header="false").drop('_c1')

In [3]:
actual = [col for col in df_train.columns]
new = ['id', 'cycle', 'setting1', 'setting2', 'setting3', 's1','s2', 's3', 's4', 's5', 's6', 's7', 's8', 's9', 's10', 's11', 's12', 's13', 's14', 's15', 's16', 's17', 's18', 's19', 's20', 's21']
for i in range(len(new)):
  df_train = df_train.withColumnRenamed(actual[i],new[i])
  df_test = df_test.withColumnRenamed(actual[i],new[i])
df_truth = df_truth.withColumnRenamed('_c0','ttf')

In [4]:
maximum = df_train.groupby('id').agg({'cycle':'max'}).withColumnRenamed('id','Ids').withColumnRenamed('max(cycle)','Max')
#maximum.show(100)
step1 = df_train.join(maximum, df_train.id == maximum.Ids,how='left').drop('Ids')# Join 'left_outer'
#step1.show(5)
#genrate time to failure after how many stages = max cycle - cycle
step2 = step1.withColumn("ttf", step1.Max-step1.cycle).drop('Max')
#step2.show(5)

In [5]:
from pyspark.ml.feature import Bucketizer, VectorAssembler, MinMaxScaler
from pyspark.ml import Pipeline
bucketizer = [Bucketizer(splits=[-float("inf"),30,float("inf")], inputCol="ttf", outputCol= "binary")]
bucketizer += [Bucketizer(splits=[-float("inf"),15,30,60,float("inf")], inputCol="ttf", outputCol= "ternary")]

assembler = [VectorAssembler(inputCols= [col], outputCol="Vect_"+col)  for col in step2.columns if col not in ['id','cycle','ttf']]
minmax = [MinMaxScaler(inputCol="Vect_"+col, outputCol='scaled_'+col) for col in step2.columns if col not in ['id','cycle','ttf']]
assembler1 = VectorAssembler(inputCols= ['scaled_'+col for col in step2.columns if col not in ['ttf','id','cycle']]+['id','cycle'], outputCol="features")  
stage = bucketizer + assembler + minmax+ [assembler1]
stepf = Pipeline(stages = stage).fit(step2).transform(step2).select(*['scaled_'+col for col in step2.columns if col not in ['id','cycle','ttf','binary','ternary']]+['id','cycle','features','ttf','binary','ternary'])

In [6]:
train1 = stepf.select('features','ttf').withColumnRenamed('ttf','label')
train2 = stepf.select('features','binary').withColumnRenamed('binary','label')
train3 = stepf.select('features','ternary').withColumnRenamed('ternary','label')

In [7]:
import pyspark.sql.functions as sparkf
#df_test = df_test.drop('setting3','s1','s5','s10','s16','s18','s19')
truth = df_truth.withColumn('Ids', sparkf.monotonically_increasing_id()+1).select('Ids','ttf')
maximum = df_test.groupby('id').agg({'cycle':'max'}).withColumnRenamed('id','Ids').withColumnRenamed('max(cycle)','Max')
step1 = df_test.join(maximum, df_test.id == maximum.Ids,how='left').drop('Ids')# Join 'left_outer'
step1 = step1[step1['cycle'] == step1['Max']].drop('Max')
test = step1.join(truth, step1.id == truth.Ids,how='left').drop('Ids')# Join 'left_outer'
test.show()
stept = Pipeline(stages = stage).fit(test).transform(test).select(*['scaled_'+col for col in test.columns if col not in ['id','cycle','ttf','binary','ternary']]+['id','cycle','features','ttf','binary','ternary'])

+---+-----+--------+--------+--------+------+------+-------+-------+-----+-----+------+-------+-------+---+-----+------+-------+-------+------+----+---+----+-----+-----+-------+---+
 id|cycle|setting1|setting2|setting3| s1| s2| s3| s4| s5| s6| s7| s8| s9|s10| s11| s12| s13| s14| s15| s16|s17| s18| s19| s20| s21|ttf|
+---+-----+--------+--------+--------+------+------+-------+-------+-----+-----+------+-------+-------+---+-----+------+-------+-------+------+----+---+----+-----+-----+-------+---+
 1| 31| -6.0E-4| 4.0E-4| 100.0|518.67|642.58|1581.22|1398.91|14.62|21.61|554.42|2388.08| 9056.4|1.3|47.23|521.79|2388.06|8130.11|8.4024|0.03|393|2388|100.0|38.81|23.3552|112|
 2| 49| 0.0018| -1.0E-4| 100.0|518.67|642.55|1586.59|1410.83|14.62|21.61|553.52| 2388.1|9044.77|1.3|47.67|521.74|2388.09| 8126.9|8.4505|0.03|391|2388|100.0|38.81|23.2618| 98|
 3| 126| -0.0016| 4.0E-4| 100.0|518.67|642.88|1589.75|1418.89|14.62|21.61|552.59|2388.16|9049.26|1.3|47.88|520.83|2388.14|8131.46|8.4119|0.03|395|2388|100.0|38.93| 23.274| 69|
 4| 106| 0.0012| 4.0E-4| 100.0|518.67|642.78|1594.53|1406.88|14.62|21.61|552.64|2388.13| 9051.3|1.3|47.65|521.88|2388.11|8133.64|8.4634|0.03|395|2388|100.0|38.58|23.2581| 82|
 5| 98| -0.0013| -4.0E-4| 100.0|518.67|642.27|1589.94|1419.36|14.62|21.61|553.29| 2388.1|9053.99|1.3|47.46| 521.0|2388.15|8125.74|8.4362|0.03|394|2388|100.0|38.75|23.4117| 91|
 6| 105| 0.0076| -3.0E-4| 100.0|518.67|643.05|1586.94|1404.49|14.62|21.61|553.73|2388.14|9055.83|1.3|47.51|521.56|2388.15|8139.02|8.4452|0.03|393|2388|100.0|38.91|23.3269| 93|
 7| 160| 0.0016| -1.0E-4| 100.0|518.67| 642.1|1589.59|1413.57|14.62|21.61|553.42|2388.04|9070.99|1.3|47.32|522.32|2388.07|8149.19|8.4028|0.03|393|2388|100.0|38.91|23.2763| 91|
 8| 166| 0.0016| -5.0E-4| 100.0|518.67|642.59|1591.91|1413.89|14.62|21.61|553.51|2388.14|9049.96|1.3|47.59|520.53|2388.12|8127.84|8.4744|0.03|393|2388|100.0|38.73|23.2465| 95|
 9| 55| -3.0E-4| 4.0E-4| 100.0|518.67|642.27|1593.61|1410.27|14.62|21.61|552.58|2388.13|9057.57|1.3|47.58|521.65|2388.16|8134.76|8.4293|0.03|392|2388|100.0|38.78|23.3515|111|
 10| 192| -0.0018| 4.0E-4| 100.0|518.67| 643.0| 1589.5|1398.99|14.62|21.61|552.88|2388.11|9060.88|1.3|47.59|521.82|2388.08|8141.91|8.4194|0.03|393|2388|100.0|38.79|23.2956| 96|
 11| 83| 1.0E-4| 4.0E-4| 100.0|518.67|642.42|1587.73|1414.22|14.62|21.61| 552.8|2388.21|9037.11|1.3|47.67|520.74|2388.18|8116.85| 8.464|0.03|394|2388|100.0|38.66|23.2908| 97|
 12| 217| 0.0013| 3.0E-4| 100.0|518.67|642.59| 1595.9|1409.64|14.62|21.61|553.36|2388.04|9074.24|1.3|47.37|522.03|2388.02|8155.74|8.4482|0.03|393|2388|100.0|38.81|23.2861|124|
 13| 195| 0.0032| -4.0E-4| 100.0|518.67|642.37|1591.31|1414.51|14.62|21.61|553.28| 2388.1|9060.92|1.3|47.52| 521.3|2388.12| 8146.8|8.4336|0.03|394|2388|100.0|38.75| 23.282| 95|
 14| 46| 0.0013| -2.0E-4| 100.0|518.67|642.91|1591.39|1408.35|14.62|21.61|553.75|2388.17|9037.35|1.3|47.57|520.66| 2388.1| 8118.1|8.4233|0.03|393|2388|100.0|38.69|23.2603|107|
 15| 76| -0.0012| -0.0| 100.0|518.67|642.49|1589.21|1400.61|14.62|21.61|554.08| 2388.1| 9056.1|1.3|47.25|522.12|2388.11|8139.75|8.4067|0.03|392|2388|100.0|39.17|23.4085| 83|
 16| 113| -5.0E-4| 2.0E-4| 100.0|518.67|642.99|1587.03| 1401.0|14.62|21.61| 554.6|2388.08|9059.75|1.3|47.37|521.77|2388.08| 8147.4|8.4299|0.03|391|2388|100.0|38.81| 23.304| 84|
 17| 165| 9.0E-4| -1.0E-4| 100.0|518.67| 642.9|1599.17|1412.54|14.62|21.61|553.49|2388.06|9094.72|1.3|47.55|521.52|2388.03|8171.98|8.4319|0.03|394|2388|100.0| 38.7|23.2948| 50|
 18| 133| 7.0E-4| 1.0E-4| 100.0|518.67| 643.0|1600.45|1419.18|14.62|21.61|553.36|2388.14|9056.09|1.3| 47.7|521.06|2388.19| 8136.8|8.4418|0.03|395|2388|100.0|38.71|23.2637| 28|
 19| 135| 4.0E-4| 3.0E-4| 100.0|518.67|642.79|1588.88|1406.32|14.62|21.61|553.87| 2388.1|9055.54|1.3|47.35|521.48|2388.06| 8140.2|8.4768|0.03|393|2388|100.0|38.83|23.3297| 87|
 20| 184| -6.0E-4| -4.0E-4| 100.0|518.67|643.75|1595.12| 1419.6|14.62|21.61|552.19|2388.14|9066.49|1.3|48.13|520.64|2388.21|8138.86| 8.469|0.03|395|2388|100.0|

In [8]:
test1 = stept.select('features','ttf').withColumnRenamed('ttf','label')
test2 = stept.select('features','binary').withColumnRenamed('binary','label')
test3 = stept.select('features','ternary').withColumnRenamed('ternary','label')

# * MACHINE LEARNING

In [10]:
trainDF = train2
testDF = test2

In [11]:
from pyspark.ml.classification import LogisticRegression

# Load training data


lr = LogisticRegression()

# Fit the model
lrModel = lr.fit(trainDF)

# Print the coefficients and intercept for logistic regression
print("Coefficients: " + str(lrModel.coefficients))
print("Intercept: " + str(lrModel.intercept))
lrModel.summary.accuracy

Coefficients: [-0.3850788772155179,-0.11803541116077872,0.0,0.0,-2.232610698799144,-0.8107970972647518,-3.462570053939408,0.0,2.1911687233330728,3.818809014702152,-8.143856251345806,-9.437428021535384,0.0,-7.04358873416446,4.704994644587844,-7.644975416803354,-6.780607319463963,-2.5813978535154494,0.0,-1.7891958367564524,0.0,0.0,1.9940176886431211,2.4070568895982998,0.0018487930384252225,0.011320666776354179]
Intercept: 11.703687140076651
Out[10]: 0.9629198778537152

In [12]:
#for test df
result = lrModel.transform(testDF)
result.select("prediction","label","features").show(10)

+----------+-----+--------------------+
prediction|label| features|
+----------+-----+--------------------+
 1.0| 1.0|[0.39259259259259...|
 1.0| 1.0|[0.57037037037037...|
 0.0| 1.0|[0.31851851851851...|
 1.0| 1.0|[0.52592592592592...|
 1.0| 1.0|[0.34074074074074...|
 1.0| 1.0|[1.0,0.2000000000...|
 1.0| 1.0|[0.55555555555555...|
 1.0| 1.0|[0.55555555555555...|
 1.0| 1.0|[0.41481481481481...|
 1.0| 1.0|[0.30370370370370...|
+----------+-----+--------------------+
only showing top 10 rows

In [13]:
from pyspark.ml.evaluation import BinaryClassificationEvaluator

evaluator = BinaryClassificationEvaluator()
print("AUC: %(result)s" % {"result": evaluator.evaluate(result)})
display(lrModel, trainDF, "ROC")

False Positive Rate,True Positive Rate,Threshold
0.0,0.0,0.9999993808025834
0.0,0.011235955056179775,0.9999993808025834
0.0,0.02247191011235955,0.9999993647332793
0.0,0.033707865168539325,0.9999992472719322
0.0,0.0449438202247191,0.9999988588844383
0.0,0.056179775280898875,0.999998526772801
0.0,0.06741573033707865,0.9999983401197873
0.0,0.07865168539325842,0.999998272305648
0.0,0.0898876404494382,0.9999980411658588
0.0,0.10112359550561797,0.9999976765232945


In [14]:
from pyspark.ml.evaluation import BinaryClassificationEvaluator

# Let's use the run-of-the-mill evaluator
evaluator = BinaryClassificationEvaluator(labelCol='label')

# We have only two choices: area under ROC and PR curves :-(
auroc = evaluator.evaluate(result, {evaluator.metricName: "areaUnderROC"})
auprc = evaluator.evaluate(result, {evaluator.metricName: "areaUnderPR"})
print("Area under ROC Curve: {:.4f}".format(auroc))
print("Area under PR Curve: {:.4f}".format(auprc))

Area under ROC Curve: 0.9771
Area under PR Curve: 0.9924

In [15]:
from pyspark.ml.evaluation import BinaryClassificationEvaluator
lr.setRegParam(0.08)

model = lr.fit(trainDF)
#training = model.transform(trainDF)
result = model.transform(testDF)
print("evaluations %(result)s" % {"result": BinaryClassificationEvaluator().evaluate(result)})

evaluations 0.9792000000000001

In [16]:
from pyspark.mllib.evaluation import MulticlassMetrics
from pyspark.sql.types import FloatType
import pyspark.sql.functions as F
#important: need to cast to float type, and order by prediction, else it won't work
preds_and_labels = result.select(['prediction','label']).withColumn('label', F.col('label').cast(FloatType())).orderBy('prediction')
#select only prediction and label columns
preds_and_labels = preds_and_labels.select(['prediction','label'])

metrics = MulticlassMetrics(preds_and_labels.rdd.map(tuple))

print(metrics.confusionMatrix().toArray())


[[24. 1.]
 [ 8. 67.]]

In [17]:
# We are trying for all possible evaluations possible

In [18]:
from pyspark.ml.classification import LogisticRegression
logr = LogisticRegression(featuresCol='features', labelCol='label')


In [19]:
from pyspark.ml.tuning import ParamGridBuilder
param_grid = ParamGridBuilder().\
      addGrid(logr.regParam, [0, 0.1, 0.2, 0.5]).\
      addGrid(logr.elasticNetParam, [0, 0.1, 0.2, 0.5]).\
      addGrid(logr.maxIter, [5,10,20]).\
      build()

In [20]:
from pyspark.ml.evaluation import BinaryClassificationEvaluator
lrevaluator = BinaryClassificationEvaluator(rawPredictionCol="rawPrediction")

In [21]:
from pyspark.ml.tuning import CrossValidator
cv = CrossValidator(estimator=logr, evaluator=evaluator, estimatorParamMaps=param_grid, numFolds=3)
cv_model = cv.fit(trainDF)  # fitiing data to my cross validation model

/databricks/spark/python/pyspark/ml/util.py:791: UserWarning: Can not find mlflow. To enable mlflow logging, install MLflow library from PyPi.
 warnings.warn(_MLflowInstrumentation._NO_MLFLOW_WARNING)

In [22]:
show_columns = ['features', 'label', 'prediction', 'rawPrediction', 'probability']
pred_training_lr = cv_model.transform(trainDF)
pred_training_lr.select(show_columns).show(5, truncate=False)

+-----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+-----+----------+----------------------------------------+------------------------------------------+
features |label|prediction|rawPrediction |probability |
+-----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+-----+----------+----------------------------------------+------------------------------------------+
[0.4597701149425288,0.16666666666666663,0.5,0.5,0.18373493975904379,0.40680183126226366,0.30975692099932317,0.5,1.0,0.7262479871175599,0.2424242424240754,0.10975500314098934,0.5,0.36904761904761757,0.6332622601279213,0.2058823529410388,0.19960780266281528,0.36398614851866024,0.5,0.3333333333333333,0.5,0.5,0.7131782945736452,0.7246616956641801,1.0,1.0]|1.0 |1.0 |[-10.889879892120472,10.889879892120472]|[1.864563406020456E-5,0.9999813543659397] |
[0.6091954022988506,0.25,0.5,0.5,0.28313253012046957,0.45301940265968865,0.35263335584064986,0.5,1.0,0.6280193236715017,0.21212121212106597,0.10024230458583903,0.5,0.3809523809523814,0.7654584221748323,0.27941176470602985,0.16281349984518642,0.41131204309350256,0.5,0.3333333333333333,0.5,0.5,0.6666666666666666,0.7310135321734316,1.0,2.0] |1.0 |1.0 |[-10.616552058893136,10.616552058893136]|[2.4506392267038153E-5,0.999975493607733] |
[0.2528735632183908,0.75,0.5,0.5,0.3433734939759061,0.36952256376716813,0.3705266711681304,0.5,1.0,0.7101449275362356,0.2727272727270848,0.14004307637081964,0.5,0.25000000000000105,0.7953091684434864,0.2205882352943045,0.17179275467024466,0.35744517121970065,0.5,0.16666666666666666,0.5,0.5,0.6279069767441883,0.6213753106876546,1.0,3.0] |1.0 |1.0 |[-11.248851120027435,11.248851120027435]|[1.3022080488669393E-5,0.9999869779195114]|
[0.5402298850574713,0.5,0.5,0.5,0.3433734939759061,0.2561587093961188,0.3311951384199846,0.5,1.0,0.7407407407407536,0.31818181818194347,0.12451763438930237,0.5,0.16666666666666738,0.8891257995735633,0.29411764705862686,0.17488904943750838,0.16660253943824677,0.5,0.3333333333333333,0.5,0.5,0.5736434108527151,0.6623860811930374,1.0,4.0] |1.0 |1.0 |[-12.299425185856032,12.299425185856032]|[4.554340880183028E-6,0.9999954456591198] |
[0.39080459770114945,0.3333333333333333,0.5,0.5,0.3493975903614429,0.2574667538696297,0.40462525320729276,0.5,1.0,0.6682769726248033,0.2424242424240754,0.14995961590236015,0.5,0.2559523809523808,0.7462686567164273,0.23529411764690147,0.1747342346991466,0.40207772220084576,0.5,0.4166666666666667,0.5,0.5,0.5891472868217043,0.704501518917423,1.0,5.0] |1.0 |1.0 |[-10.305216211964854,10.305216211964854]|[3.3456995188889446E-5,0.9999665430048111]|
+-----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+-----+----------+----------------------------------------+------------------------------------------+
only showing top 5 rows

In [23]:
pred_test_lr = cv_model.transform(testDF)
pred_test_lr.select(show_columns).show(5, truncate=False)

+----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+-----+----------+------------------------------------------+------------------------------------------+
features |label|prediction|rawPrediction |probability |
+----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+-----+----------+------------------------------------------+------------------------------------------+
[0.3925925925925926,0.8999999999999999,0.5,0.5,0.3640350877193205,0.1401045068582619,0.24507546687132564,0.5,1.0,0.8448687350835126,0.3000000000003032,0.23029498985484576,0.5,0.14285714285714093,0.7320872274143195,0.30303030303009426,0.1446063549888687,0.19122462344465987,0.5,0.42857142857142855,0.5,0.5,0.547619047619051,0.6983122362869185,1.0,31.0] |1.0 |1.0 |[-11.814793590901427,11.814793590901427] |[7.394301032282755E-6,0.9999926056989676] |
[0.5703703703703704,0.4,0.5,0.5,0.3508771929824404,0.3154800783801401,0.5500127909951373,0.5,1.0,0.6300715990453346,0.36666666666686876,0.13953488372093684,0.5,0.5619047619047629,0.7165109034267944,0.39393939393981153,0.11212305201376199,0.5062213490504228,0.5,0.14285714285714285,0.5,0.5,0.547619047619051,0.5012658227848109,2.0,49.0] |1.0 |1.0 |[-6.404055017932506,6.404055017932506] |[0.0016520993218994543,0.9983479006781006]|
[0.31851851851851853,0.8999999999999999,0.5,0.5,0.4956140350877232,0.41868060091443443,0.7562036326426212,0.5,1.0,0.40811455847255707,0.5666666666665656,0.1745746839394458,0.5,0.7619047619047629,0.4330218068535887,0.5454545454541696,0.15826755717466479,0.25343811394891147,0.5,0.7142857142857143,0.5,0.5,0.6904761904761915,0.5270042194092834,3.0,126.0]|1.0 |0.0 |[0.19760097824542133,-0.19760097824542133]|[0.5492401289560971,0.4507598710439029] |
[0.5259259259259259,0.8999999999999999,0.5,0.5,0.45175438596490575,0.5747877204441533,0.4489639293937098,0.5,1.0,0.4200477326968897,0.4666666666674751,0.1904947713438397,0.5,0.5428571428571409,0.7601246105919002,0.4545454545458304,0.1803278688524658,0.5907007203667305,0.5,0.7142857142857143,0.5,0.5,0.2738095238095213,0.49345991561181085,4.0,106.0] |1.0 |1.0 |[-2.4460209169511975,2.4460209169511975] |[0.0797300192077685,0.9202699807922315] |
[0.34074074074074073,0.09999999999999999,0.5,0.5,0.22807017543859123,0.42488569562377654,0.7682271680736725,0.5,1.0,0.5751789976133501,0.36666666666686876,0.21148743561729438,0.5,0.3619047619047629,0.4859813084112097,0.575757575757868,0.10038453754300852,0.412573673870328,0.5,0.5714285714285714,0.5,0.5,0.4761904761904766,0.8175105485232049,5.0,98.0] |1.0 |1.0 |[-4.759691657630424,4.759691657630424] |[0.008495459714300642,0.9915045402856992] |
+----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+-----+----------+------------------------------------------+------------------------------------------+
only showing top 5 rows

In [24]:
print('Intercept: ' + str(cv_model.bestModel.intercept) + "\n" 'coefficients: ' + str(cv_model.bestModel.coefficients))

Intercept: 1.8968594722537568
coefficients: [0.08156196669097047,-0.003371018270295428,0.0,0.0,-2.1061475804317364,-0.35955849747988583,-3.105241139145101,0.0,5.32156742655121,5.823033457433,-6.079292767826093,-8.473917020244466,0.0,-5.407250852069497,6.555059598287182,-5.878824317962665,-6.7601662787178505,-2.510822051325166,0.0,-1.4812210532593841,0.0,0.0,3.341957278643953,3.7516614745816814,0.0014054590400560034,0.01281526623788717]

In [25]:
print('Logistic Regression', "\n",'The best RegParam is: ', cv_model.bestModel._java_obj.getRegParam(), "\n",'The best ElasticNetParam is:', cv_model.bestModel._java_obj.getElasticNetParam(), "\n",'The best Iteration is:',cv_model.bestModel._java_obj.getMaxIter() , "\n", 'Area under ROC is:', cv_model.bestModel.summary.areaUnderROC)

Logistic Regression 
 The best RegParam is: 0.0 
 The best ElasticNetParam is: 0.0 
 The best Iteration is: 20 
 Area under ROC is: 0.9896208855614165

In [26]:
lrevaluator.evaluate(pred_test_lr, {lrevaluator.metricName: "areaUnderROC"})
lrevaluator.evaluate(pred_test_lr, {lrevaluator.metricName: "areaUnderPR"})

Out[48]: 0.9933299465667366

In [27]:
from pyspark.ml.classification import LinearSVC

# Load training data


lsvc = LinearSVC(maxIter=10, regParam=0.1)

# Fit the model
lsvcModel = lsvc.fit(trainDF)

# Print the coefficients and intercept for linear SVC
print("Coefficients: " + str(lsvcModel.coefficients))
print("Intercept: " + str(lsvcModel.intercept))

lsvcresult = lsvcModel.transform(testDF)
lsvcresult.select("prediction","label","features").show(10)

#Compute accuracy of test
from pyspark.ml.evaluation import BinaryClassificationEvaluator
evaluator = BinaryClassificationEvaluator()
print("evaluation: %(result)s" % {"result": evaluator.evaluate(lsvcresult)})

Coefficients: [0.20267675567233134,0.05389551084763348,0.0,0.0,-0.09386870521068165,-0.0745487731929049,-0.09183273191330549,0.0,0.2002768331656202,0.47014932097171724,0.023360105819224545,-0.09594496155934235,0.0,-0.07488003890507051,0.39655277902336933,0.0385353736169196,-0.0,-0.10461253910512425,0.0,-0.08318689655346187,0.0,0.0,0.4486344608414418,0.4114359197456823,0.00039307470635352123,-0.00026639952682968335]
Intercept: 0.006732123185823175
+----------+-----+--------------------+
prediction|label| features|
+----------+-----+--------------------+
 1.0| 1.0|[0.39259259259259...|
 1.0| 1.0|[0.57037037037037...|
 1.0| 1.0|[0.31851851851851...|
 1.0| 1.0|[0.52592592592592...|
 1.0| 1.0|[0.34074074074074...|
 1.0| 1.0|[1.0,0.2000000000...|
 1.0| 1.0|[0.55555555555555...|
 1.0| 1.0|[0.55555555555555...|
 1.0| 1.0|[0.41481481481481...|
 1.0| 1.0|[0.30370370370370...|
+----------+-----+--------------------+
only showing top 10 rows

evaluation: 0.9733333333333334

In [28]:
from pyspark.ml.evaluation import BinaryClassificationEvaluator

# Let's use the run-of-the-mill evaluator
svmevaluator = BinaryClassificationEvaluator()

# We have only two choices: area under ROC and PR curves :-(
svmauroc = svmevaluator.evaluate(lsvcresult, {svmevaluator.metricName: "areaUnderROC"})
svmauprc = svmevaluator.evaluate(lsvcresult, {svmevaluator.metricName: "areaUnderPR"})
print("Area under ROC Curve: {:.4f}".format(svmauroc))
print("Area under PR Curve: {:.4f}".format(svmauprc))

Area under ROC Curve: 0.9733
Area under PR Curve: 0.9912

In [29]:
#ESTIMATOR
from pyspark.ml.classification import LinearSVC
lsvm = LinearSVC(featuresCol='features', labelCol='label')

#GRID VECTOR

from pyspark.ml.tuning import ParamGridBuilder
param_grid_svm = ParamGridBuilder().\
      addGrid(lsvm.regParam, [0, 0.1, 0.2, 0.5]).\
      addGrid(lsvm.maxIter, [5,10,20,50]).\
      build()

#Evaluator
from pyspark.ml.evaluation import BinaryClassificationEvaluator
svmevaluator = BinaryClassificationEvaluator(rawPredictionCol="rawPrediction")

#CROSS VALIDATION
from pyspark.ml.tuning import CrossValidator
cv_svm = CrossValidator(estimator=lsvm, evaluator=svmevaluator, estimatorParamMaps=param_grid_svm, numFolds=3)
cv_svm_model = cv_svm.fit(trainDF)  # fitiing data to my cross validation model

show_columns = ['features', 'label', 'prediction', 'rawPrediction']
pred_training_svm = cv_svm_model.transform(trainDF)
pred_training_svm.select(show_columns).show(5, truncate=False)

pred_test_svm = cv_svm_model.transform(testDF)
pred_test_svm.select(show_columns).show(5, truncate=False)

print('Support Vector Machine', "\n",'The best RegParam is: ', cv_svm_model.bestModel._java_obj.getRegParam(),  "\n",'The best Iteration is:',cv_svm_model.bestModel._java_obj.getMaxIter() , "\n", 'Area under ROC is:', svmevaluator.evaluate(pred_test_svm, {svmevaluator.metricName: "areaUnderROC"}))

+-----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+-----+----------+----------------------------------------+
features |label|prediction|rawPrediction |
+-----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+-----+----------+----------------------------------------+
[0.4597701149425288,0.16666666666666663,0.5,0.5,0.18373493975904379,0.40680183126226366,0.30975692099932317,0.5,1.0,0.7262479871175599,0.2424242424240754,0.10975500314098934,0.5,0.36904761904761757,0.6332622601279213,0.2058823529410388,0.19960780266281528,0.36398614851866024,0.5,0.3333333333333333,0.5,0.5,0.7131782945736452,0.7246616956641801,1.0,1.0]|1.0 |1.0 |[-3.3904642220360945,3.3904642220360945]|
[0.6091954022988506,0.25,0.5,0.5,0.28313253012046957,0.45301940265968865,0.35263335584064986,0.5,1.0,0.6280193236715017,0.21212121212106597,0.10024230458583903,0.5,0.3809523809523814,0.7654584221748323,0.27941176470602985,0.16281349984518642,0.41131204309350256,0.5,0.3333333333333333,0.5,0.5,0.6666666666666666,0.7310135321734316,1.0,2.0] |1.0 |1.0 |[-3.3204633141420046,3.3204633141420046]|
[0.2528735632183908,0.75,0.5,0.5,0.3433734939759061,0.36952256376716813,0.3705266711681304,0.5,1.0,0.7101449275362356,0.2727272727270848,0.14004307637081964,0.5,0.25000000000000105,0.7953091684434864,0.2205882352943045,0.17179275467024466,0.35744517121970065,0.5,0.16666666666666666,0.5,0.5,0.6279069767441883,0.6213753106876546,1.0,3.0] |1.0 |1.0 |[-3.3317293203849156,3.3317293203849156]|
[0.5402298850574713,0.5,0.5,0.5,0.3433734939759061,0.2561587093961188,0.3311951384199846,0.5,1.0,0.7407407407407536,0.31818181818194347,0.12451763438930237,0.5,0.16666666666666738,0.8891257995735633,0.29411764705862686,0.17488904943750838,0.16660253943824677,0.5,0.3333333333333333,0.5,0.5,0.5736434108527151,0.6623860811930374,1.0,4.0] |1.0 |1.0 |[-3.714655384606793,3.714655384606793] |
[0.39080459770114945,0.3333333333333333,0.5,0.5,0.3493975903614429,0.2574667538696297,0.40462525320729276,0.5,1.0,0.6682769726248033,0.2424242424240754,0.14995961590236015,0.5,0.2559523809523808,0.7462686567164273,0.23529411764690147,0.1747342346991466,0.40207772220084576,0.5,0.4166666666666667,0.5,0.5,0.5891472868217043,0.704501518917423,1.0,5.0] |1.0 |1.0 |[-3.096891122590129,3.096891122590129] |
+-----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+-----+----------+----------------------------------------+
only showing top 5 rows

+----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+-----+----------+----------------------------------------+
features |label|prediction|rawPrediction |
+------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

In [30]:
from pyspark.ml.classification import NaiveBayes
from pyspark.ml.evaluation import BinaryClassificationEvaluator

# create the trainer and set its parameters
nb = NaiveBayes(smoothing=1.0, modelType="multinomial")

# train the model
nbmodel = nb.fit(trainDF)

# select example rows to display.
nbresult = model.transform(testDF)
nbresult.select("prediction","label","features").show(10)

# compute accuracy on the test set
nbevaluator = BinaryClassificationEvaluator()
accuracy = nbevaluator.evaluate(nbresult)
print("evaluations: %(nbresult)s" % {"nbresult": nbevaluator.evaluate(nbresult)})

+----------+-----+--------------------+
prediction|label| features|
+----------+-----+--------------------+
 1.0| 1.0|[0.39259259259259...|
 1.0| 1.0|[0.57037037037037...|
 1.0| 1.0|[0.31851851851851...|
 1.0| 1.0|[0.52592592592592...|
 1.0| 1.0|[0.34074074074074...|
 1.0| 1.0|[1.0,0.2000000000...|
 1.0| 1.0|[0.55555555555555...|
 1.0| 1.0|[0.55555555555555...|
 1.0| 1.0|[0.41481481481481...|
 1.0| 1.0|[0.30370370370370...|
+----------+-----+--------------------+
only showing top 10 rows

evaluations: 0.9792000000000001

In [31]:
from pyspark.ml.evaluation import BinaryClassificationEvaluator

# Let's use the run-of-the-mill evaluator
nbevaluator = BinaryClassificationEvaluator()

# We have only two choices: area under ROC and PR curves :-(
nbauroc = nbevaluator.evaluate(nbresult, {nbevaluator.metricName: "areaUnderROC"})
nbauprc = nbevaluator.evaluate(nbresult, {nbevaluator.metricName: "areaUnderPR"})
print("Area under ROC Curve: {:.4f}".format(nbauroc))
print("Area under PR Curve: {:.4f}".format(nbauprc))

Area under ROC Curve: 0.9792
Area under PR Curve: 0.9932

In [32]:
#ESTIMATOR
from pyspark.ml.classification import NaiveBayes
nb = NaiveBayes(featuresCol='features', labelCol='label')

#GRID VECTOR

from pyspark.ml.tuning import ParamGridBuilder
param_grid_nb = ParamGridBuilder().\
      addGrid(nb.smoothing, [0, 0.5, 1,2]).\
      addGrid(nb.modelType, ["multinomial"]).\
      build()

#Evaluator
from pyspark.ml.evaluation import BinaryClassificationEvaluator
nbevaluator = BinaryClassificationEvaluator(rawPredictionCol="rawPrediction")

#CROSS VALIDATION
from pyspark.ml.tuning import CrossValidator
cv_nb = CrossValidator(estimator=nb, evaluator=nbevaluator, estimatorParamMaps=param_grid_nb, numFolds=3)
cv_nb_model = cv_nb.fit(trainDF)  # fitiing data to my cross validation model

show_columns = ['features', 'label', 'prediction', 'rawPrediction', 'probability']
pred_training_nb = cv_nb_model.transform(trainDF)
pred_training_nb.select(show_columns).show(5, truncate=False)

pred_test_nb = cv_nb_model.transform(testDF)
pred_test_nb.select(show_columns).show(5, truncate=False)

print('Naive Bayes ',"\n",'The best Smoothening is: ', cv_nb_model.bestModel._java_obj.getSmoothing(), "\n",'The best model type is:', cv_nb_model.bestModel._java_obj.getModelType(), "\n", 'Area under ROC is:', nbevaluator.evaluate(pred_test_nb, {nbevaluator.metricName: "areaUnderROC"}))

##nbauroc = nbevaluator.evaluate(nbresult, {pred_test_nb.metricName: "areaUnderROC"})
#nbauprc = nbevaluator.evaluate(nbresult, {pred_test_nb.metricName: "areaUnderPR"})


+-----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+-----+----------+---------------------------------------+------------------------------------------+
features |label|prediction|rawPrediction |probability |
+-----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+-----+----------+---------------------------------------+------------------------------------------+
[0.4597701149425288,0.16666666666666663,0.5,0.5,0.18373493975904379,0.40680183126226366,0.30975692099932317,0.5,1.0,0.7262479871175599,0.2424242424240754,0.10975500314098934,0.5,0.36904761904761757,0.6332622601279213,0.2058823529410388,0.19960780266281528,0.36398614851866024,0.5,0.3333333333333333,0.5,0.5,0.7131782945736452,0.7246616956641801,1.0,1.0]|1.0 |1.0 |[-70.30637492664563,-62.89093873980308]|[6.015277460127192E-4,0.9993984722539873] |
[0.6091954022988506,0.25,0.5,0.5,0.28313253012046957,0.45301940265968865,0.35263335584064986,0.5,1.0,0.6280193236715017,0.21212121212106597,0.10024230458583903,0.5,0.3809523809523814,0.7654584221748323,0.27941176470602985,0.16281349984518642,0.41131204309350256,0.5,0.3333333333333333,0.5,0.5,0.6666666666666666,0.7310135321734316,1.0,2.0] |1.0 |1.0 |[-73.43686659059033,-66.13666626854062]|[6.749476014874776E-4,0.9993250523985125] |
[0.2528735632183908,0.75,0.5,0.5,0.3433734939759061,0.36952256376716813,0.3705266711681304,0.5,1.0,0.7101449275362356,0.2727272727270848,0.14004307637081964,0.5,0.25000000000000105,0.7953091684434864,0.2205882352943045,0.17179275467024466,0.35744517121970065,0.5,0.16666666666666666,0.5,0.5,0.6279069767441883,0.6213753106876546,1.0,3.0] |1.0 |1.0 |[-72.56182076679019,-65.46240200914033]|[8.249036208295229E-4,0.9991750963791706] |
[0.5402298850574713,0.5,0.5,0.5,0.3433734939759061,0.2561587093961188,0.3311951384199846,0.5,1.0,0.7407407407407536,0.31818181818194347,0.12451763438930237,0.5,0.16666666666666738,0.8891257995735633,0.29411764705862686,0.17488904943750838,0.16660253943824677,0.5,0.3333333333333333,0.5,0.5,0.5736434108527151,0.6623860811930374,1.0,4.0] |1.0 |1.0 |[-72.93950660639598,-65.90946081864735]|[8.841089222291409E-4,0.9991158910777709] |
[0.39080459770114945,0.3333333333333333,0.5,0.5,0.3493975903614429,0.2574667538696297,0.40462525320729276,0.5,1.0,0.6682769726248033,0.2424242424240754,0.14995961590236015,0.5,0.2559523809523808,0.7462686567164273,0.23529411764690147,0.1747342346991466,0.40207772220084576,0.5,0.4166666666666667,0.5,0.5,0.5891472868217043,0.704501518917423,1.0,5.0] |1.0 |1.0 |[-72.44414891365095,-65.96816486732847]|[0.0015376148766689736,0.9984623851233311]|
+-----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+-----+----------+---------------------------------------+------------------------------------------+
only showing top 5 rows

+----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+----

In [34]:
from pyspark.ml.classification import RandomForestClassifier

from pyspark.ml.evaluation import BinaryClassificationEvaluator

# Train a RandomForest model.
rf = RandomForestClassifier( numTrees=10)


# Train model.  This also runs the indexers.
rfmodel = rf.fit(trainDF)

# Make predictions.
rfresult = rfmodel.transform(testDF)

# Select example rows to display.
rfresult.select("prediction","label","features").show(10)

# Select (prediction, true label) and compute test error
rfevaluator = BinaryClassificationEvaluator()
print("evaluations: %(rfresult)s" % {"rfresult": rfevaluator.evaluate(rfresult)})



+----------+-----+--------------------+
prediction|label| features|
+----------+-----+--------------------+
 1.0| 1.0|[0.39259259259259...|
 1.0| 1.0|[0.57037037037037...|
 1.0| 1.0|[0.31851851851851...|
 1.0| 1.0|[0.52592592592592...|
 1.0| 1.0|[0.34074074074074...|
 1.0| 1.0|[1.0,0.2000000000...|
 1.0| 1.0|[0.55555555555555...|
 1.0| 1.0|[0.55555555555555...|
 1.0| 1.0|[0.41481481481481...|
 1.0| 1.0|[0.30370370370370...|
+----------+-----+--------------------+
only showing top 10 rows

evaluations: 0.9829333333333333

In [35]:
# We have only two choices: area under ROC and PR curves :-(
rfauroc = rfevaluator.evaluate(rfresult, {rfevaluator.metricName: "areaUnderROC"})
rfauprc = rfevaluator.evaluate(rfresult, {rfevaluator.metricName: "areaUnderPR"})
print("Area under ROC Curve: {:.4f}".format(rfauroc))
print("Area under PR Curve: {:.4f}".format(rfauprc))

Area under ROC Curve: 0.9829
Area under PR Curve: 0.9945

In [36]:
#ESTIMATOR
from pyspark.ml.classification import RandomForestClassifier
from pyspark.ml.evaluation import BinaryClassificationEvaluator

# Train a RandomForest model.
rf = RandomForestClassifier(featuresCol='features', labelCol='label')

#GRID VECTOR
from pyspark.ml.tuning import ParamGridBuilder
param_grid_rf = ParamGridBuilder().addGrid(rf.maxDepth, [2, 3, 4]).\
      addGrid(rf.minInfoGain, [0.0, 0.1, 0.2, 0.3]).\
      addGrid(rf.numTrees,[10,20,30,40,50,60, 100]).\
      build()
 
#Evaluator
from pyspark.ml.evaluation import BinaryClassificationEvaluator
rfevaluator = BinaryClassificationEvaluator(rawPredictionCol="rawPrediction")

#CROSS VALIDATION
from pyspark.ml.tuning import CrossValidator
cv_rf = CrossValidator(estimator=rf, evaluator=rfevaluator, estimatorParamMaps=param_grid_rf, numFolds=3)
cv_rf_model = cv_rf.fit(trainDF)  # fitiing data to my cross validation model

show_columns = ['features', 'label', 'prediction', 'rawPrediction', 'probability']
pred_training_rf = cv_rf_model.transform(trainDF)
pred_training_rf.select(show_columns).show(5, truncate=False)

pred_test_rf = cv_rf_model.transform(testDF)
pred_test_rf.select(show_columns).show(5, truncate=False)

print('Random forest ',"\n",'The best Max Depth is: ', cv_rf_model.bestModel._java_obj.getMaxDepth(), "\n",'The best min Info gain is:', cv_rf_model.bestModel._java_obj.getMinInfoGain(), "\n", 'Area under ROC is:', rfevaluator.evaluate(pred_test_rf, {rfevaluator.metricName: "areaUnderROC"}))
#rfmodel.trees
##nbauroc = rfevaluator.evaluate(pred_test_rf, {rfevaluator.metricName: "areaUnderROC"})
#nbauprc = rfevaluator.evaluate(pred_test_rf, {rfevaluator.metricName: "areaUnderPR"})

+-----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+-----+----------+--------------------------------------+-----------------------------------------+
features |label|prediction|rawPrediction |probability |
+-----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+-----+----------+--------------------------------------+-----------------------------------------+
[0.4597701149425288,0.16666666666666663,0.5,0.5,0.18373493975904379,0.40680183126226366,0.30975692099932317,0.5,1.0,0.7262479871175599,0.2424242424240754,0.10975500314098934,0.5,0.36904761904761757,0.6332622601279213,0.2058823529410388,0.19960780266281528,0.36398614851866024,0.5,0.3333333333333333,0.5,0.5,0.7131782945736452,0.7246616956641801,1.0,1.0]|1.0 |1.0 |[1.2952611114225483,98.70473888857748]|[0.012952611114225479,0.9870473888857745]|
[0.6091954022988506,0.25,0.5,0.5,0.28313253012046957,0.45301940265968865,0.35263335584064986,0.5,1.0,0.6280193236715017,0.21212121212106597,0.10024230458583903,0.5,0.3809523809523814,0.7654584221748323,0.27941176470602985,0.16281349984518642,0.41131204309350256,0.5,0.3333333333333333,0.5,0.5,0.6666666666666666,0.7310135321734316,1.0,2.0] |1.0 |1.0 |[1.2952611114225483,98.70473888857748]|[0.012952611114225479,0.9870473888857745]|
[0.2528735632183908,0.75,0.5,0.5,0.3433734939759061,0.36952256376716813,0.3705266711681304,0.5,1.0,0.7101449275362356,0.2727272727270848,0.14004307637081964,0.5,0.25000000000000105,0.7953091684434864,0.2205882352943045,0.17179275467024466,0.35744517121970065,0.5,0.16666666666666666,0.5,0.5,0.6279069767441883,0.6213753106876546,1.0,3.0] |1.0 |1.0 |[1.2952611114225483,98.70473888857748]|[0.012952611114225479,0.9870473888857745]|
[0.5402298850574713,0.5,0.5,0.5,0.3433734939759061,0.2561587093961188,0.3311951384199846,0.5,1.0,0.7407407407407536,0.31818181818194347,0.12451763438930237,0.5,0.16666666666666738,0.8891257995735633,0.29411764705862686,0.17488904943750838,0.16660253943824677,0.5,0.3333333333333333,0.5,0.5,0.5736434108527151,0.6623860811930374,1.0,4.0] |1.0 |1.0 |[1.2952611114225483,98.70473888857748]|[0.012952611114225479,0.9870473888857745]|
[0.39080459770114945,0.3333333333333333,0.5,0.5,0.3493975903614429,0.2574667538696297,0.40462525320729276,0.5,1.0,0.6682769726248033,0.2424242424240754,0.14995961590236015,0.5,0.2559523809523808,0.7462686567164273,0.23529411764690147,0.1747342346991466,0.40207772220084576,0.5,0.4166666666666667,0.5,0.5,0.5891472868217043,0.704501518917423,1.0,5.0] |1.0 |1.0 |[1.2952611114225483,98.70473888857748]|[0.012952611114225479,0.9870473888857745]|
+-----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+-----+----------+--------------------------------------+-----------------------------------------+
only showing top 5 rows

+----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+-----+----------+---

In [38]:
from pyspark.ml.classification import GBTClassifier
from pyspark.ml.evaluation import BinaryClassificationEvaluator

# Train a GBT model.
gb = GBTClassifier()
# Chain indexers and GBT in a Pipeline

# Train model.  This also runs the indexers.
gbmodel = gb.fit(trainDF)

# Make predictions.
gbresult = gbmodel.transform(testDF)

# Select example rows to display.
gbresult.select("prediction","label","features").show(5)

# Select (prediction, true label) and compute test error
gbevaluator = BinaryClassificationEvaluator()

print("evaluations: %(gbresult)s" % {"gbresult": gbevaluator.evaluate(gbresult)})


+----------+-----+--------------------+
prediction|label| features|
+----------+-----+--------------------+
 1.0| 1.0|[0.39259259259259...|
 1.0| 1.0|[0.57037037037037...|
 0.0| 1.0|[0.31851851851851...|
 1.0| 1.0|[0.52592592592592...|
 1.0| 1.0|[0.34074074074074...|
+----------+-----+--------------------+
only showing top 5 rows

evaluations: 0.9589333333333336

In [39]:
# We have only two choices: area under ROC and PR curves :-(
gbauroc = gbevaluator.evaluate(gbresult, {gbevaluator.metricName: "areaUnderROC"})
gbauprc = gbevaluator.evaluate(gbresult, {gbevaluator.metricName: "areaUnderPR"})
print("Area under ROC Curve: {:.4f}".format(gbauroc))
print("Area under PR Curve: {:.4f}".format(gbauprc))

Area under ROC Curve: 0.9589
Area under PR Curve: 0.9875

In [40]:
from pyspark.ml.classification import GBTClassifier
from pyspark.ml.evaluation import BinaryClassificationEvaluator

# ESTIMATOR
gbt = GBTClassifier(featuresCol='features', labelCol='label')


#GRID VECTOR
from pyspark.ml.tuning import ParamGridBuilder
param_grid_gbt = ParamGridBuilder().\
    addGrid(gbt.maxDepth, [2, 3]).\
    addGrid(gbt.minInfoGain, [0.0, 0.1]).\
    addGrid(gbt.stepSize, [0.02, 0.05]).\
    build()
 
#Evaluator
from pyspark.ml.evaluation import BinaryClassificationEvaluator
gbtevaluator = BinaryClassificationEvaluator(rawPredictionCol="rawPrediction")

#CROSS VALIDATION
from pyspark.ml.tuning import CrossValidator
cv_gbt = CrossValidator(estimator=gbt, evaluator=gbtevaluator, estimatorParamMaps=param_grid_gbt)
cv_gbt_model = cv_gbt.fit(trainDF)  # fitiing data to my cross validation model

show_columns = ['features', 'label', 'prediction', 'rawPrediction', 'probability']
pred_training_gbt = cv_gbt_model.transform(trainDF)
pred_training_gbt.select(show_columns).show(5, truncate=False)

pred_test_gbt = cv_gbt_model.transform(testDF)
pred_test_gbt.select(show_columns).show(5, truncate=False)


print('Gradient Boosting ',"\n",'The best Max Depth is: ', cv_gbt_model.bestModel._java_obj.getMaxDepth(), "\n",'The best min Info gain is:',cv_gbt_model.bestModel._java_obj.getMinInfoGain(), "\n", 'step size: ', cv_gbt_model.bestModel._java_obj.getStepSize(),"\n" ,'Area under ROC is:', rfevaluator.evaluate(pred_test_rf, {rfevaluator.metricName: "areaUnderROC"}))

+-----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+-----+----------+----------------------------------------+---------------------------------------+
features |label|prediction|rawPrediction |probability |
+-----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+-----+----------+----------------------------------------+---------------------------------------+
[0.4597701149425288,0.16666666666666663,0.5,0.5,0.18373493975904379,0.40680183126226366,0.30975692099932317,0.5,1.0,0.7262479871175599,0.2424242424240754,0.10975500314098934,0.5,0.36904761904761757,0.6332622601279213,0.2058823529410388,0.19960780266281528,0.36398614851866024,0.5,0.3333333333333333,0.5,0.5,0.7131782945736452,0.7246616956641801,1.0,1.0]|1.0 |1.0 |[-1.3198205398552647,1.3198205398552647]|[0.0666303536252356,0.9333696463747644]|
[0.6091954022988506,0.25,0.5,0.5,0.28313253012046957,0.45301940265968865,0.35263335584064986,0.5,1.0,0.6280193236715017,0.21212121212106597,0.10024230458583903,0.5,0.3809523809523814,0.7654584221748323,0.27941176470602985,0.16281349984518642,0.41131204309350256,0.5,0.3333333333333333,0.5,0.5,0.6666666666666666,0.7310135321734316,1.0,2.0] |1.0 |1.0 |[-1.3198205398552647,1.3198205398552647]|[0.0666303536252356,0.9333696463747644]|
[0.2528735632183908,0.75,0.5,0.5,0.3433734939759061,0.36952256376716813,0.3705266711681304,0.5,1.0,0.7101449275362356,0.2727272727270848,0.14004307637081964,0.5,0.25000000000000105,0.7953091684434864,0.2205882352943045,0.17179275467024466,0.35744517121970065,0.5,0.16666666666666666,0.5,0.5,0.6279069767441883,0.6213753106876546,1.0,3.0] |1.0 |1.0 |[-1.3198205398552647,1.3198205398552647]|[0.0666303536252356,0.9333696463747644]|
[0.5402298850574713,0.5,0.5,0.5,0.3433734939759061,0.2561587093961188,0.3311951384199846,0.5,1.0,0.7407407407407536,0.31818181818194347,0.12451763438930237,0.5,0.16666666666666738,0.8891257995735633,0.29411764705862686,0.17488904943750838,0.16660253943824677,0.5,0.3333333333333333,0.5,0.5,0.5736434108527151,0.6623860811930374,1.0,4.0] |1.0 |1.0 |[-1.3198205398552647,1.3198205398552647]|[0.0666303536252356,0.9333696463747644]|
[0.39080459770114945,0.3333333333333333,0.5,0.5,0.3493975903614429,0.2574667538696297,0.40462525320729276,0.5,1.0,0.6682769726248033,0.2424242424240754,0.14995961590236015,0.5,0.2559523809523808,0.7462686567164273,0.23529411764690147,0.1747342346991466,0.40207772220084576,0.5,0.4166666666666667,0.5,0.5,0.5891472868217043,0.704501518917423,1.0,5.0] |1.0 |1.0 |[-1.3198205398552647,1.3198205398552647]|[0.0666303536252356,0.9333696463747644]|
+-----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+-----+----------+----------------------------------------+---------------------------------------+
only showing top 5 rows

+----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+-----+----------+---

In [41]:
from pyspark.mllib.evaluation import MulticlassMetrics
scoreandlabels = gbresult.rdd.map(lambda z: (z["label"], z["prediction"]))
metric = MulticlassMetrics(scoreandlabels)


In [42]:
#model_stages = cv + cv_svm + cv_nb+cv_rf +cv_gbt
#pipelined_models = Pipeline(stages= model_stages).fit(trainDF)
#pipelined_result = pipelined_models.transform(testDF)

In [43]:
print('Models and their Performance',"\n")
print('Logistic Regression',cv_model.bestModel.summary.areaUnderROC)
print('Support Vector Machine',svmevaluator.evaluate(pred_test_svm, {svmevaluator.metricName: "areaUnderROC"}))
print('Naive Bayes', nbevaluator.evaluate(pred_test_nb, {nbevaluator.metricName: "areaUnderROC"}))
print('Random forest', rfevaluator.evaluate(pred_test_rf, {rfevaluator.metricName: "areaUnderROC"}))
print('Gradient Boost', gbtevaluator.evaluate(pred_test_gbt, {rfevaluator.metricName: "areaUnderROC"}))

Models and their Performance 

Logistic Regression 0.9896208855614165
Support Vector Machine 0.9813333333333334
Naive Bayes 0.7557333333333331
Random forest 0.9754666666666667
Gradient Boost 0.9642666666666667

In [44]:
from pyspark.sql.types import *
schema = StructType([StructField('model', StringType(), False),StructField('train AUC', DoubleType(), True),StructField('test AUC', DoubleType(), True),StructField('train PR', DoubleType(), True),StructField('test PR', DoubleType(), True)])
df = sqlContext.createDataFrame(sc.emptyRDD(), schema)

list = ['lr', 'svm', 'nb','rf','gbt']
columns = df.columns
for model in list:
  evaluators = eval(model+"evaluator")
  training_set = eval("pred_training_"+model)
  test_set = eval("pred_test_"+model)
  trnau = evaluators.evaluate(training_set, {evaluators.metricName: "areaUnderROC"})
  tstau = evaluators.evaluate(test_set, {evaluators.metricName: "areaUnderROC"})
  
  trnpr = evaluators.evaluate(test_set, {evaluators.metricName: "areaUnderPR"})
  tstpr = evaluators.evaluate(test_set, {evaluators.metricName: "areaUnderPR"})
  
  newRow = spark.createDataFrame([(model,trnau,tstau,trnpr,tstpr)], columns)
  df = df.union(newRow)
  #appended.show()
  

In [45]:
df.show()

+-----+------------------+------------------+------------------+------------------+
model| train AUC| test AUC| train PR| test PR|
+-----+------------------+------------------+------------------+------------------+
 lr|0.9896613351483171|0.9802666666666666|0.9933299465667366|0.9933299465667366|
 svm|0.9880457149339203|0.9813333333333334|0.9936930274185863|0.9936930274185863|
 nb|0.8020261282211265|0.7557333333333331|0.9094330350208518|0.9094330350208518|
 rf| 0.988535080256355|0.9754666666666667|0.9917419135847867|0.9917419135847867|
 gbt|0.9854601365019914|0.9642666666666667|0.9882893700661207|0.9882893700661207|
+-----+------------------+------------------+------------------+------------------+